In [1]:
import nltk
import glob
import os
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import pickle5 as pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rupin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rupin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
PATH = 'CSE508_Winter2023_Dataset/CSE508_Winter2023_Dataset/'
PATH_upd = 'CSE508_Winter2023_Dataset_upd/CSE508_Winter2023_Dataset/'
flist = os.listdir(PATH)

In [3]:
for i in flist:
    s= open(PATH+i).read()
    ans = s[s.index('<TITLE>')+8:s.index('</TITLE>')].strip() + " " + s[s.index('<TEXT>')+7:s.index('</TEXT>')].strip()
    open(PATH_upd+i, 'w').write(ans)


In [4]:
def rem_sw(arr):
  global stop_words
  reg_str = r'^'
  for i in range(len(arr)):
    for sw in stop_words:
      reg_str+=sw
      reg_str+='$'
      arr[i] = re.sub(reg_str, '', arr[i])
      reg_str = r'^'
  return arr

def rem_punc(str):
  str = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\/)', ' ', str)
  return str

In [6]:
for i in flist:
    s = open(PATH_upd+i, 'r').read()
    s = s.lower()
    open(PATH_upd+i, 'w').write(s)

In [7]:
def preprocessing(s):
    s = s.lower()
    s = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\/)', ' ', s)
    s = word_tokenize(s)
    s = rem_sw(s)
    for j in range(len(s)):
      s[j] = rem_punc(s[j])
    s[:] = [j for j in s if (j!='' and j!=' ')]
    return s


In [8]:
for i in flist:
    s = open(PATH_upd+i, 'r').read()
    s = s.lower()
    s = rem_punc(s)
    s = word_tokenize(s)
    s = rem_sw(s)
    for j in range(len(s)):
      s[j] = rem_punc(s[j])
    s[:] = [j for j in s if (j!='' and j!=' ')]
    open(PATH_upd+i, 'w').write(','.join(s))

In [11]:
# pos_index structure: {term : [df, {doc_id : [tf, [pos1, pos2, ...]]}]} df = frequency in all docs combined, tf = frequency in a particular doc, pos[] = positions of term in doc
pos_index = {}
for doc_id in flist:
    f = open(PATH_upd+doc_id, 'r')
    tokens = f.read().split(',')
    for num,j in enumerate(tokens):
        if j not in set(pos_index.keys()):  #new term
            pos_index[j] = []
            pos_index[j].append(1)
            pos_index[j].append({doc_id:[1,[num]]})
        else:
            pos_index[j][0]+=1
            if doc_id not in set(pos_index[j][1].keys()):   #new document for this term
                pos_index[j][1][doc_id] = [1]
                pos_index[j][1][doc_id].append([num])
            else:
                pos_index[j][1][doc_id][1].append(num)
                pos_index[j][1][doc_id][0]+=1
    f.close()


In [12]:
with open('pos_index.pickle', 'wb') as f:
    pickle.dump(pos_index, f, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:

with open('pos_index.pickle', 'rb') as f:
    pos_index = pickle.load(f)


In [10]:
query = input()
query = preprocessing(query)

In [11]:
print(query)

['experimental', 'investigation']


In [12]:
term_freq = []
for term in query:
    if term in set(pos_index.keys()):
        term_freq.append(pos_index[term][0])
    else:
        term_freq.append(0)
si = [sorted_index[0] for sorted_index in sorted(enumerate(term_freq),key=lambda i:i[1])]
sorted_query = [query[i] for i in si]
print(sorted_query)
print(term_freq)

['investigation', 'experimental']
[423, 281]


In [13]:
common_docs = list(pos_index[sorted_query[0]][1].keys())
for term in range(1,len(sorted_query)):
    if term in set(pos_index.keys()):
        common_docs = [i for i in common_docs if i in set(pos_index[term][1].keys())]
print(len(common_docs))

216


In [16]:
query

['experimental', 'investigation']

In [17]:
ans_docs = {}   # stores the possible documents with list of possible phrase start positions
for n, term in enumerate(query):
    if n==0:    # for the first term
        #ans_docs = pos_index[term][1]
        for i in list(set(pos_index[term][1].keys())):
            ans_docs[i] = pos_index[term][1][i][1].copy()
        #print(ans_docs['cranfield1201'])
        #for i in list(set(pos_index[term][1].keys())):
            #ans_docs[i] = pos_index[term][1][i]
    else:
        to_check_docs = list(set(pos_index[term][1].keys()).intersection(set(ans_docs.keys())))
        #print(to_check_docs)
        for doc in to_check_docs:
            temp = []
            for pos in pos_index[term][1][doc][1]:
                if doc=='cranfield1201':
                    #print(pos-n)
                    pass
                if pos-n in set(ans_docs[doc]):
                    
                    temp.append(pos-n)
                    #ans_docs[doc].remove(pos-n)
            """for i in ans_docs[doc]:
                if doc=='cranfield1201':
                    print(i)
                    print(ans_docs['cranfield1201'])
                if i not in set(temp):
                    ans_docs[doc].remove(i)"""
            ans_docs[doc] = [i for i in ans_docs[doc] if i in set(temp)]
                
            if len(ans_docs[doc]) ==0:
                del ans_docs[doc]
        for doc in list(set(ans_docs.keys()).difference(set(to_check_docs))):
            del ans_docs[doc]
        if len(ans_docs) == 0:
            print("No documents found")
            break
    

In [18]:
print(len(list(ans_docs.keys())))

50


In [194]:
pos_index['flow'][1]['cranfield0002']

[6, [2, 14, 34, 46, 89, 115]]